In [1]:
from pykrx import stock
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.metrics import MeanSquaredError
import matplotlib.pyplot as plt

#ticker_list=[]
# 빈 리스트
results_list = []

# 각 종목에 대해 반복
for ticker in tqdm(ticker_list, desc="Processing"):
    try:
        ticker_name = stock.get_market_ticker_name(ticker)
        start_date = "2020-03-20"
        end_date = "2024-03-28"
        # 주어진 기간 동안의 일별 거래량 정보 가져오기
        df = stock.get_market_ohlcv_by_date(fromdate=start_date, todate=end_date, ticker=ticker)


        def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
            """MACD 및 MACD 신호 계산"""
            df['EMA_short'] = df['종가'].ewm(span=short_window, adjust=False).mean()
            df['EMA_long'] = df['종가'].ewm(span=long_window, adjust=False).mean()
            df['MACD'] = df['EMA_short'] - df['EMA_long']
            df['MACD_Signal'] = df['MACD'].ewm(span=signal_window, adjust=False).mean()
            return df

        def calculate_rsi(df, window=14, signal_window=9):
            """RSI 및 RSI 신호 계산"""
            delta = df['종가'].diff()
            gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()

            rs = gain / loss
            df['RSI'] = 100 - (100 / (1 + rs))
            
            # RSI 신호선 추가
            df['RSI_Signal'] = df['RSI'].ewm(span=signal_window, adjust=False).mean()
            return df

        # MACD 및 MACD 신호 계산
        df = calculate_macd(df)

        df = calculate_rsi(df)


        # 지정된 기간에 대한 SMA 계산
        periods = [5, 20, 60, 120, 240]
        for period in periods:
            df[f'SMA_{period}'] = df['종가'].rolling(window=period).mean()

        # 지정된 기간에 대한 EMA 계산
        for period in periods:
            df[f'EMA_{period}'] = df['종가'].ewm(span=period, adjust=False).mean()

        for i in range(len(periods)):
            for j in range(i + 1, len(periods)):
                smaller_period = periods[i]
                larger_period = periods[j]
                df[f'SMA_{smaller_period}_minus_SMA_{larger_period}'] = df[f'SMA_{smaller_period}'] - df[f'SMA_{larger_period}']

        # EMA 간의 차이 계산
        for i in range(len(periods)):
            for j in range(i + 1, len(periods)):
                smaller_period = periods[i]
                larger_period = periods[j]
                df[f'EMA_{smaller_period}_minus_EMA_{larger_period}'] = df[f'EMA_{smaller_period}'] - df[f'EMA_{larger_period}']

        for column in ['시가', '고가', '저가','거래량','MACD','MACD_Signal','RSI','RSI_Signal']:
            df[f'{column}_등락률'] = df[column].pct_change() * 100

        # SMA 및 EMA의 전 거래일 대비 등락률 계산 및 DataFrame에 추가
        periods = [5, 20, 60, 120, 240]

        # SMA 등락률 계산 및 추가
        for period in periods:
            df[f'SMA_{period}_등락률'] = df[f'SMA_{period}'].pct_change() * 100

        # EMA 등락률 계산 및 추가
        for period in periods:
            df[f'EMA_{period}_등락률'] = df[f'EMA_{period}'].pct_change() * 100

        # 'MACD'가 0 이상일 때 1, 아니면 0을 할당
        df['MACD_Binary'] = (df['MACD'] >= 0).astype(int)

        # 'MACD - MACD_Signal'이 0 이상일 때 1, 아니면 0을 할당
        df['MACD_minus_Signal_Binary'] = ((df['MACD'] - df['MACD_Signal']) >= 0).astype(int)

        # 'RSI - RSI_Signal'이 0 이상일 때 1, 아니면 0을 할당
        df['RSI_minus_Signal_Binary'] = ((df['RSI'] - df['RSI_Signal']) >= 0).astype(int)

        # 시가 대비 종가 등락률 컬럼 추가
        df['시가_대비_종가_등락률'] = ((df['종가'] - df['시가']) / df['시가']) * 100
        df['시가_대비_저가_등락률'] = ((df['저가'] - df['시가']) / df['시가']) * 100
        df['시가_대비_고가_등락률'] = ((df['고가'] - df['시가']) / df['시가']) * 100
        df['저가_대비_종가_등락률'] = ((df['종가'] - df['저가']) / df['저가']) * 100
        df['저가_대비_고가_등락률'] = ((df['고가'] - df['저가']) / df['저가']) * 100
        df['고가_대비_종가_등락률'] = ((df['종가'] - df['고가']) / df['고가']) * 100
        # 종가 - SMA [5, 20, 60, 120, 240] 값의 차이 컬럼 추가
        for period in [5, 20, 60, 120, 240]:
            df[f'종가_minus_SMA_{period}'] = df['종가'] - df[f'SMA_{period}']

        # 종가 - EMA [5, 20, 60, 120, 240] 값의 차이 컬럼 추가
        for period in [5, 20, 60, 120, 240]:
            df[f'종가_minus_EMA_{period}'] = df['종가'] - df[f'EMA_{period}']


        df['next_day_return'] = df['등락률'].shift(-1)

        df1=df.dropna()  # 마지막 행 삭제

        first_column_name = df.columns[0]

        #df1= df.iloc[300:, :]
        X = df1.drop(['next_day_return'], axis=1)
        y = df1['next_day_return']


        # 시계열 데이터와 타겟 생성 함수
        def generate_time_series_data(n_samples, n_steps):
            freq1, freq2, offsets1, offsets2 = np.random.rand(4, n_samples, 1)
            time = np.linspace(0, 1, n_steps)
            series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))  # 웨이브 패턴 1
            series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20))  # 웨이브 패턴 2
            series += 0.1 * (np.random.rand(n_samples, n_steps) - 0.5)  # 잡음 추가
            return series[..., np.newaxis].astype(np.float32)

        # 데이터셋 생성
        n_steps = 50
        series = generate_time_series_data(10000, n_steps + 1)
        X, y = series[:, :-1], series[:, -1]
        X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)

        # LSTM 모델 구축
        model = Sequential([
            LSTM(50, return_sequences=True, input_shape=[None, 1]),
            LSTM(50),
            Dense(1)
        ])

        model.compile(optimizer='adam', loss='mean_squared_error', metrics=[MeanSquaredError()])
        first_column_name = df.columns[0]
        X_last = df.iloc[-1:,:]

        y_pred1 = model.predict(X_last)

        results_list.append({
            'Ticker': ticker,
            'Name': ticker_name,
            '등락률': y_pred1
        })

        pass
    except Exception as e:
        print(f"Error processing {ticker}: {e}")
        continue

# 결과를 데이터프레임으로 변환하고 precision_svm 값에 따라 내림차순 정렬
results_df = pd.DataFrame(results_list).sort_values(by='등락률', ascending=False)



KeyboardInterrupt

